In [1]:
from itertools import chain

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import yahooquery as yq

from funcs.loaders import *

In [2]:
pd.set_option('plotting.backend', "plotly")

In [3]:
msci_world_gross = read_msci_data('data/MSCI/WORLD/STANDARD/BLEND/MSCI WORLD USD Gross Monthly.xls')

In [4]:
msci_world_net = read_msci_data('data/MSCI/WORLD/STANDARD/BLEND/MSCI World USD Net Monthly.xls')

In [5]:
msci_world = msci_world_gross.merge(msci_world_net, on='date').pct_change().mean(axis=1).rename('price').reset_index().set_index('date').add(1).cumprod().mul(100).fillna(100)

In [6]:
sti = read_sti_data()

In [7]:
sti_1m = sti.resample('BM').last().iloc[:-1]

C:\Users\leowy\AppData\Local\Temp\ipykernel_25852\2868932432.py:1: FutureWarning: 'BM' is deprecated and will be removed in a future version, please use 'BME' instead.
  sti_1m = sti.resample('BM').last().iloc[:-1]


In [8]:
fed_funds_rate, fed_funds_rate_1m = load_fed_funds_rate()

c:\Users\leowy\_Projects\Stock-Market-Returns\funcs\loaders.py:60: FutureWarning: 'BM' is deprecated and will be removed in a future version, please use 'BME' instead.
  fed_funds_rate_1m = fed_funds_rate.div(36000).add(1).resample('BM').prod().pow(12).sub(1).mul(100)


In [9]:
sp500_gross = read_spx_data('Gross')

In [10]:
sp500_net = read_spx_data('Net')

In [11]:
shiller_sp500 = read_shiller_sp500_data('Net')

In [12]:
usdsgd = load_usdsgd()

In [13]:
usd_fx = await load_fred_usd_fx_async()

In [14]:
sgd_fx = load_mas_sgd_fx()

In [15]:
sgd_interest_rates, sgd_interest_rates_1m = load_sgd_interest_rates()

c:\Users\leowy\_Projects\Stock-Market-Returns\funcs\loaders.py:402: FutureWarning: 'BM' is deprecated and will be removed in a future version, please use 'BME' instead.
  sgd_interest_rates_1m = sgd_interest_rates.resample('D').ffill().div(36500).add(1).resample('BM').prod().pow(12).sub(1).mul(100).replace(0, np.nan)


In [16]:
sgd_neer = load_sgd_neer()

In [17]:
usd_sgd_swap_points = load_mas_swap_points()

c:\Users\leowy\_Projects\Stock-Market-Returns\funcs\loaders.py:316: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df['date'] = df['month'] - MonthEnd() + pd.TimedeltaIndex(df['day'], unit='D')
c:\Users\leowy\_Projects\Stock-Market-Returns\funcs\loaders.py:316: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df['date'] = df['month'] - MonthEnd() + pd.TimedeltaIndex(df['day'], unit='D')


In [18]:
sg_cpi = load_sg_cpi()

In [19]:
us_cpi = await load_us_cpi_async()

In [20]:
msci_world = msci_world.merge(fed_funds_rate_1m, left_index=True, right_index=True, how='left')

In [21]:
msci_world = msci_world.merge(sgd_interest_rates_1m['sgd_ir_1m'], left_index=True, right_index=True, how='left')

In [22]:
periods = ['1m', '3m', '6m', '1y', '2y', '3y', '5y', '10y', '15y', '20y', '25y', '30y']
durations = [1, 3, 6, 12, 24, 36, 60, 120, 180, 240, 300, 360]

In [23]:
add_return_columns(msci_world, periods, durations)

In [24]:
add_return_columns(sti_1m, periods, durations)

In [25]:
add_return_columns(shiller_sp500, periods, durations)

In [26]:
msci_world.head(10)

,price,ffr,sgd_ir_1m,1m_cumulative,3m_cumulative,6m_cumulative,1y_cumulative,2y_cumulative,3y_cumulative,5y_cumulative,...,6m_annualized,1y_annualized,2y_annualized,3y_annualized,5y_annualized,10y_annualized,15y_annualized,20y_annualized,25y_annualized,30y_annualized
date,,,,,,,,,,,,,,,,,,,,,
1969-12-31,100.000000,10.391636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-01-30,94.491500,9.376079,NaN,-0.055085,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-27,97.481485,8.771530,NaN,0.031643,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,97.827451,8.657911,NaN,0.003549,-0.021725,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,88.727406,8.432125,NaN,-0.093021,-0.061001,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-29,83.173339,7.988168,NaN,-0.062597,-0.146778,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-06-30,81.167249,8.469048,NaN,-0.024119,-0.170302,-0.188328,NaN,NaN,NaN,NaN,...,-0.341188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-07-31,86.244611,7.739472,NaN,0.062554,-0.027982,-0.087277,NaN,NaN,NaN,NaN,...,-0.166936,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-08-31,89.130958,7.074040,NaN,0.033467,0.071629,-0.085663,NaN,NaN,NaN,NaN,...,-0.163987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
msci_world.describe()

,price,ffr,sgd_ir_1m,1m_cumulative,3m_cumulative,6m_cumulative,1y_cumulative,2y_cumulative,3y_cumulative,5y_cumulative,...,6m_annualized,1y_annualized,2y_annualized,3y_annualized,5y_annualized,10y_annualized,15y_annualized,20y_annualized,25y_annualized,30y_annualized
count,653.000000,653.000000,438.000000,652.000000,650.000000,647.000000,641.000000,629.000000,617.000000,593.000000,...,647.000000,641.000000,629.000000,617.000000,593.000000,533.000000,473.000000,413.000000,353.000000,293.000000
mean,2722.041942,5.198582,1.729499,0.008380,0.025776,0.052878,0.109109,0.226750,0.356368,0.662352,...,0.121767,0.109109,0.100698,0.098262,0.096734,0.097967,0.097877,0.095922,0.095568,0.095136
std,2959.835902,4.254962,1.645635,0.043106,0.077263,0.115042,0.171206,0.272774,0.364748,0.604905,...,0.240135,0.171206,0.123444,0.098019,0.074127,0.047357,0.040737,0.032132,0.024923,0.016508
min,81.167249,0.048345,0.016769,-0.189471,-0.331688,-0.434646,-0.469420,-0.470921,-0.453187,-0.246839,...,-0.680375,-0.469420,-0.272622,-0.182264,-0.055118,-0.023284,0.030301,0.034885,0.058008,0.067044
25%,288.295079,1.512627,0.308281,-0.016278,-0.012007,-0.011228,0.012418,0.080983,0.174653,0.238783,...,-0.022329,0.012418,0.039703,0.055123,0.043756,0.068287,0.063562,0.068129,0.071642,0.078793
50%,1812.486604,5.191583,1.300574,0.012082,0.029274,0.057720,0.127695,0.246868,0.335271,0.597525,...,0.118772,0.127695,0.116632,0.101175,0.098220,0.092379,0.088155,0.091561,0.092857,0.096299
75%,3834.424788,7.336620,2.874909,0.033280,0.073249,0.116020,0.203928,0.371005,0.531314,0.888279,...,0.245501,0.203928,0.170899,0.152625,0.135568,0.129898,0.141588,0.124348,0.116389,0.107932
max,13069.631063,23.069445,8.023426,0.146480,0.306124,0.469370,0.665528,1.388766,2.002092,3.358870,...,1.159047,0.665528,0.545563,0.442585,0.342378,0.207729,0.181283,0.158232,0.153991,0.130163


In [28]:
msci_world.loc[:, '1m_annualized':'30y_annualized'].describe()

,1m_annualized,3m_annualized,6m_annualized,1y_annualized,2y_annualized,3y_annualized,5y_annualized,10y_annualized,15y_annualized,20y_annualized,25y_annualized,30y_annualized
count,652.000000,650.000000,647.000000,641.000000,629.000000,617.000000,593.000000,533.000000,473.000000,413.000000,353.000000,293.000000
mean,0.236141,0.144080,0.121767,0.109109,0.100698,0.098262,0.096734,0.097967,0.097877,0.095922,0.095568,0.095136
std,0.617142,0.331369,0.240135,0.171206,0.123444,0.098019,0.074127,0.047357,0.040737,0.032132,0.024923,0.016508
min,-0.919606,-0.800512,-0.680375,-0.469420,-0.272622,-0.182264,-0.055118,-0.023284,0.030301,0.034885,0.058008,0.067044
25%,-0.178757,-0.047169,-0.022329,0.012418,0.039703,0.055123,0.043756,0.068287,0.063562,0.068129,0.071642,0.078793
50%,0.155012,0.122338,0.118772,0.127695,0.116632,0.101175,0.098220,0.092379,0.088155,0.091561,0.092857,0.096299
75%,0.481218,0.326790,0.245501,0.203928,0.170899,0.152625,0.135568,0.129898,0.141588,0.124348,0.116389,0.107932
max,4.157034,1.910296,1.159047,0.665528,0.545563,0.442585,0.342378,0.207729,0.181283,0.158232,0.153991,0.130163


In [29]:
go.Figure(
    data=[
        go.Box(
            x=msci_world[column],
            name=column,
        )
        for column in msci_world.loc[:, '1m_annualized':'30y_annualized'].columns
    ],
    layout=go.Layout(
        height=800,
        xaxis=dict(
            tickformat='.2%',
        )
    )
)

In [30]:
treasury_rates = pd.DataFrame(
    {
        duration: load_us_treasury_rate(duration)
        for duration in ['1MO', '3MO', '6MO', '1', '2', '3', '5', '7', '10', '20', '30']
    }
)

In [31]:
treasury_rates

,1MO,3MO,6MO,1,2,3,5,7,10,20,30
date,,,,,,,,,,,
1962-01-02,NaN,NaN,NaN,3.220000,NaN,3.700000,3.880000,NaN,4.060000,4.070000,NaN
1962-01-03,NaN,NaN,NaN,3.240000,NaN,3.700000,3.870000,NaN,4.030000,4.070000,NaN
1962-01-04,NaN,NaN,NaN,3.240000,NaN,3.690000,3.860000,NaN,3.990000,4.060000,NaN
1962-01-05,NaN,NaN,NaN,3.260000,NaN,3.710000,3.890000,NaN,4.020000,4.070000,NaN
1962-01-06,NaN,NaN,NaN,3.276667,NaN,3.710000,3.896667,NaN,4.023333,4.073333,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-09,5.510000,5.460000,5.41,5.120000,4.800000,4.600000,4.470000,4.460000,4.450000,4.700000,4.600000
2024-05-10,5.510000,5.470000,5.43,5.170000,4.870000,4.650000,4.520000,4.510000,4.500000,4.740000,4.640000
2024-05-11,5.506667,5.463333,5.43,5.166667,4.863333,4.653333,4.513333,4.503333,4.493333,4.733333,4.636667
